In [1]:
from deeprankcore.dataset import GridDataset
import h5py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
hdf5_path = "tests/data/hdf5/1ATN_ppi.hdf5"
#hdf5_path = '/Users/giuliacrocioni/Desktop/docs/eScience/projects/3D-vac/deeprank-core/tests/data/hdf5/train.hdf5'
dataset = GridDataset(hdf5_path, target = 'binary')

/Users/giuliacrocioni/miniforge3/envs/deeprankcore/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


   ['tests/data/hdf5/1ATN_ppi.hdf5'] dataset                 : 100%|██████████| 1/1 [00:00<00:00, 325.75it/s, entry_name=1ATN_ppi.hdf5]


In [7]:
dataset.features_dict

{'mapped_features': ['_position_000',
  '_position_001',
  '_position_002',
  'covalent',
  'distance',
  'electrostatic',
  'same_chain',
  'vanderwaals'],
 'target_values': ['binary']}

In [14]:
df_final = pd.DataFrame()

for fname in [hdf5_path]:
    with h5py.File(fname, 'r') as f:

        entry_name = list(f.keys())[0]
        

        entry_names = [entry for entry, _ in f.items()]

        df_dict = {}
        df_dict['id'] = entry_names

        for feat_type in dataset.features_dict:
            for feat in dataset.features_dict[feat_type]:
                print(entry_name)
                print(feat_type)
                print(feat)
                print(f[entry_name][feat_type][feat])
                break
                if f[entry_name][feat_type][feat][()].ndim == 2:
                    for i in range(f[entry_name][feat_type][feat][:].shape[1]):
                        df_dict[feat + '_' + str(i)] = [f[entry_name][feat_type][feat][:][:,i] for entry_name in entry_names]
                else:
                    df_dict[feat] = [
                        f[entry_name][feat_type][feat][:]
                        if f[entry_name][feat_type][feat][()].ndim == 1
                        else f[entry_name][feat_type][feat][()] for entry_name in entry_names]

#         df = pd.DataFrame(data=df_dict)

#     df_final = pd.concat([df_final, df])

# df_final.reset_index(drop=True, inplace=True)
# self.df = df_final

residue-ppi-1ATN_1w:A-B
mapped_features
_position_000
<HDF5 group "/residue-ppi-1ATN_1w:A-B/mapped_features/_position_000" (1 members)>
residue-ppi-1ATN_1w:A-B
target_values
binary
<HDF5 dataset "binary": shape (), type "|b1">


In [6]:
df_dict

{'id': ['residue-ppi-1ATN_1w:A-B',
  'residue-ppi-1ATN_2w:A-B',
  'residue-ppi-1ATN_3w:A-B',
  'residue-ppi-1ATN_4w:A-B']}

In [3]:
dataset.get(0)

/Users/giuliacrocioni/Desktop/docs/eScience/projects/3D-vac/deeprank-core/deeprankcore/dataset.py:521: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/runner/miniforge3/conda-bld/pytorch-recipe_1654951287008/work/torch/csrc/utils/tensor_new.cpp:210.)
  data = Data(x=torch.tensor([feature_data], dtype=torch.float),


Data(x=[1, 8, 20, 20, 20], y=[1], entry_names='residue-ppi-1ATN_1w:A-B')